In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib as plt
import ast
import pickle

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time
from keras_facenet import FaceNet

model = YOLO("best.pt")
embedder = FaceNet()
bounding_boxes = []
embeddings_list = []
lables = []
CONF_THRESHOLD = 0.85
count = 0

name = input("Enter your name: ")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# start_time = time.time()
while True:
    print("inside while")
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    results = model(source=frame)
    result = results[0]
    
    for i, box in enumerate(result.boxes):
        confidence = box.conf[0]
        print("inside for")
        if confidence >= CONF_THRESHOLD:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())

            print(f"Detected Bounding Box {i+1}: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

            h, w, _ = frame.shape
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)
            print("coordinates taken")
            face = frame[y1:y2, x1:x2]

            face_resized = cv2.resize(face, (160, 160))

            face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
            mean, std = face_rgb.mean(),face_rgb.std()
            face = (face_rgb-mean)/std
            face = np.expand_dims(face, axis=0)

            embedding = embedder.embeddings(face)[0]

            bounding_boxes.append((x1, y1, x2, y2))
            embeddings_list.append(embedding)
            lables.append(name)
            count = count+1
            print(f"Face detected...{count}")
            # if(count==30):
            #     break

            # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow('YOLOv8 FacedetX', result.plot())
    if((cv2.waitKey(1) & 0xFF == ord('q')) | len(lables)%60==0):
        break

cap.release()
cv2.destroyAllWindows()
# np.savez_compressed('face_data.npz', bounding_boxes=bounding_boxes, embeddings=embeddings_list)

print("Bounding boxes and embeddings stored successfully.")


Enter your name:  Mallikarjuna


inside while

0: 480x640 1 person, 2348.4ms
Speed: 107.2ms preprocess, 2348.4ms inference, 169.4ms postprocess per image at shape (1, 3, 480, 640)
inside for
Detected Bounding Box 1: x1=266, y1=244, x2=396, y2=400
coordinates taken
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Face detected...1


inside while

0: 480x640 1 person, 451.4ms
Speed: 4.9ms preprocess, 451.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
inside for
Detected Bounding Box 1: x1=267, y1=245, x2=398, y2=402
coordinates taken
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Face detected...2
inside while

0: 480x640 1 person, 198.9ms
Speed: 6.1ms preprocess, 198.9ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
inside for
Detected Bounding Box 1: x1=267, y1=245, x2=398, y2=399
coordinates taken
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Face detected...3
inside while

0: 480x640 1 person, 239.8ms
Speed: 6.9ms preprocess, 239.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
inside for
Detected Bounding Box 1: x1=269, y1=244, x2=398, y2=398
coordinates taken
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Face detected...4
inside while

0: 480x640 1 person, 225.1ms
Speed: 4.6ms preprocess, 225.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
inside for

Saving embeddings into csv file

In [4]:
output_file = 'embeddings.csv'
with open(output_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    
    for label, embedding in zip(lables, embeddings_list):
        row = [embedding[i] for i in range(0,512)]
        row.insert(0,label)
        writer.writerow(row)

print(f'Embeddings and labels saved to {output_file}')

Embeddings and labels saved to embeddings.csv


In [76]:
# import csv
# op_file = 'embeddings.csv'
# with open(op_file, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     header = [f"Embedding{i}" for i in range(0,512)]
#     header.insert(0,'Label')
#     writer.writerow(header)
    

In [3]:
# directory = 'celeb'
# lables = []
# faces = []
# embeddings_list = []

In [86]:
for celeb in os.listdir(directory):
    path = os.path.join(directory,celeb)
    for img_file in os.listdir(path):
        img_path = os.path.join(path,img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(160,160))
        faces.append(img)
        lables.append(celeb)

In [87]:
print(len(lables))
print(len(faces))

1483
1483


In [71]:
def get_embeddings(face_img):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img,axis=0)
    y_hat = embedder.embeddings(face_img)
    return y_hat[0]

In [88]:
for img in faces:
    embeddings_list.append(get_embeddings(img))
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [90]:
len(embeddings_list)

1483

In [91]:
output_file = 'embeddings.csv'
with open(output_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    
    for label, embedding in zip(lables, embeddings_list):
        row = [embedding[i] for i in range(0,512)]
        row.insert(0,label)
        writer.writerow(row)

print(f'Embeddings and labels saved to {output_file}')

Embeddings and labels saved to embeddings.csv


In [93]:
print(f"lables:{len(lables)}")
# print(lables)

lables:1483


In [94]:
print(f"Length of each embedding: {len(embeddings_list[0])}")
print(f"Total embeddings: {len(embeddings_list)}")
print(type(embeddings_list))
print(type(embeddings_list[0]))
print(type(embeddings_list[0][5]))

Length of each embedding: 512
Total embeddings: 1483
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.float32'>


In [14]:
df = pd.read_csv('embeddings.csv')
df.shape

(2083, 513)

In [15]:
total_embeddings = []
total_labels = []
for i in range(0,df.shape[0]):
    arr = np.asarray(df.iloc[i,1:])
    total_embeddings.append(arr)
    label = df.iloc[i,0]
    total_labels.append(label)
    # total_embeddings = np.append(total_embeddings, arr)

print(len(total_embeddings))
print(type(total_embeddings))
print(type(total_embeddings[0]))
print(type(total_embeddings[0][5]))

2083
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [16]:
total_embeddings = np.asarray(total_embeddings)

In [17]:
print(total_embeddings.shape)
print(total_embeddings[0].shape)
print(len(total_labels))

(2083, 512)
(512,)
2083


In [18]:
type(total_labels)

list

In [19]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels_enc = encoder.fit_transform(total_labels)

In [10]:
type(labels_enc)

numpy.ndarray

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(total_embeddings,labels_enc,test_size=0.2,random_state=42)

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.001, 0.01, 0.1, 1]
}

# Create the SVM model
svm = SVC()

# Perform grid search to find best parameters
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters found
print(f"Best parameters: {grid_search.best_params_}")


Best parameters: {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}


In [13]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear',probability=True)

In [14]:
svm_model.fit(X_train,y_train)

SVC(kernel='linear', probability=True)

In [15]:
from sklearn.metrics import accuracy_score
y_pred = svm_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.6930455635491607

In [19]:
with open('model.pkl', 'wb') as file:
    pickle.dump(svm_model, file)

In [20]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time
from keras_facenet import FaceNet

# Load the YOLO model and FaceNet
model = YOLO("/home/neeraj/Documents/face_detx/runs-20241009T055735Z-001/runs/detect/train/weights/best.pt")
embedder = FaceNet()

CONF_THRESHOLD = 0.85  # Confidence threshold for detection

# Initialize variables
name = input("Enter your name: ")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

bounding_box = []
embedding_lst = []
label = []

count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Display the live camera feed
    cv2.imshow('YOLOv8 FacedetX', frame)

    # Wait for keypress (Spacebar to take a picture, 'q' to quit)
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):  # Spacebar pressed to take the picture
        print("Processing frame...")

        # Run YOLO model on the captured frame
        results = model(source=frame)
        result = results[0]

        for i, box in enumerate(result.boxes):
            confidence = box.conf[0]

            if confidence >= CONF_THRESHOLD:
                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())

                # Ensure bounding box is within the frame
                h, w, _ = frame.shape
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                # Crop and preprocess the face
                face = frame[y1:y2, x1:x2]
                face_resized = cv2.resize(face, (160, 160))
                face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)

                # Normalize the face image
                mean, std = face_rgb.mean(), face_rgb.std()
                face = (face_rgb - mean) / std
                face = np.expand_dims(face, axis=0)

                # Generate embeddings
                embedding = embedder.embeddings(face)[0]

                # Store bounding box, embedding, and label
                bounding_box.append((x1, y1, x2, y2))
                
                embedding_lst.append(embedding)
                label.append(name)

                count += 1
                print(f"Face detected and saved... Count: {count}")

                # Draw the bounding box on the frame (for visualization)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Show the frame with the bounding box
        cv2.imshow('YOLOv8 FacedetX', result.plot())

    elif key == ord('q'):  # Quit if 'q' is pressed or if labels length is a multiple of 30
        print("Exiting...")
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

# Save the embeddings and bounding boxes to a file (if needed)
# np.savez_compressed('face_data.npz', bounding_boxes=bounding_boxes, embeddings=embeddings_list)

print("Bounding boxes and embeddings stored successfully.")


Enter your name:  kalki


Processing frame...

0: 480x640 1 person, 2263.6ms
Speed: 392.3ms preprocess, 2263.6ms inference, 176.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Face detected and saved... Count: 1
Exiting...
Bounding boxes and embeddings stored successfully.


In [21]:
embedding = np.asarray(embedding_lst[0]).flatten()
embedding_reshaped = embedding.reshape(1, -1)
pred = svm_model.predict(embedding_reshaped)
name = encoder.inverse_transform(pred)[0]
print(pred)
print(name)

[29]
Paul praneeth


For incremental learning

In [1]:
new_embeddings = []
new_labels = []

In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
import time
from keras_facenet import FaceNet

model = YOLO("/home/neeraj/Documents/face_detx/runs-20241009T055735Z-001/runs/detect/train/weights/best.pt")
embedder = FaceNet()

CONF_THRESHOLD = 0.85

name = input("Enter your name: ")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# start_time = time.time()
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    results = model(source=frame)
    result = results[0]
    
    for i, box in enumerate(result.boxes):
        confidence = box.conf[0]

        if confidence >= CONF_THRESHOLD:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())

            # print(f"Detected Bounding Box {i+1}: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

            h, w, _ = frame.shape
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face = frame[y1:y2, x1:x2]

            face_resized = cv2.resize(face, (160, 160))

            face_rgb = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
            # mean, std = face.mean(),face.std()
            # face = (face_rgb-mean)/std
            face = np.expand_dims(face_rgb, axis=0)

            embedding = embedder.embeddings(face)[0]

            # bounding_boxes.append((x1, y1, x2, y2))
            new_embeddings.append(embedding)
            new_labels.append(name)
            count = count+1
            print(f"Face detected...{count}")
            # if(count==30):
            #     break

            # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow('YOLOv8 FacedetX', result.plot())
    if((cv2.waitKey(1) & 0xFF == ord('q')) | len(new_labels)%60==0):
        break

cap.release()
cv2.destroyAllWindows()
# np.savez_compressed('face_data.npz', bounding_boxes=bounding_boxes, embeddings=embeddings_list)

print("Bounding boxes and embeddings stored successfully.")


Enter your name:  janardhan



0: 480x640 1 person, 328.0ms
Speed: 7.4ms preprocess, 328.0ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Face detected...1

0: 480x640 1 person, 236.9ms
Speed: 6.3ms preprocess, 236.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Face detected...2

0: 480x640 1 person, 217.5ms
Speed: 7.8ms preprocess, 217.5ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Face detected...3

0: 480x640 1 person, 241.6ms
Speed: 9.4ms preprocess, 241.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Face detected...4

0: 480x640 1 person, 253.3ms
Speed: 3.1ms preprocess, 253.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Face detected...5

0: 480x640 1 person, 236.2ms
Speed: 4.4ms preprocess, 236.2ms inference, 1.3m

In [5]:
print(len(new_labels))

120


In [25]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
new_labels_enc = encoder.fit_transform(new_labels)
print(new_labels_enc)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
new_embeddings = np.asarray(new_embeddings)

In [11]:
with open('model.pkl', 'rb') as file:
    svm_model = pickle.load(file)

In [20]:
svm_model.fit(new_embeddings, new_labels_enc, classes=np.unique(labels_enc))

TypeError: BaseLibSVM.fit() got an unexpected keyword argument 'classes'